In [5]:
import requests

In [1]:
apiEndPoints = {
    "album": r"https://api.deezer.com/album/{0}",
    "artist": r"https://api.deezer.com/artist/{0}",
    "track": r"https://api.deezer.com/track/{0}"    
}

In [2]:
access_token = "frBx6ZdRjE553f9dc2a3cc9P7JptufC553f9dc2a3d00hX24wk"

In [3]:
def get_item_from_api(itemType, itemId):
    r = requests.get(url = apiEndPoints[itemType].format(itemId))
    return r.json()

In [6]:
get_item_from_api("artist", 27)

{'id': 27,
 'link': 'https://www.deezer.com/artist/27',
 'name': 'Daft Punk',
 'nb_album': 30,
 'nb_fan': 3036491,
 'picture': 'https://api.deezer.com/artist/27/image',
 'picture_big': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/500x500-000000-80-0-0.jpg',
 'picture_medium': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/250x250-000000-80-0-0.jpg',
 'picture_small': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/56x56-000000-80-0-0.jpg',
 'radio': True,
 'share': 'http://www.deezer.com/artist/27?utm_source=deezer&utm_content=artist-27&utm_term=0_1455096115&utm_medium=web',
 'tracklist': 'https://api.deezer.com/artist/27/top?limit=50',
 'type': 'artist'}

In [9]:
def get_history(processingFunction = process_history, length = 50):
    output = []
    increment = 50
    numberOfSteps = length // increment
    for step in range(numberOfSteps):
        url = r"http://api.deezer.com/user/me/history?access_token={0}&index={1}&limit={2}".format(access_token,step*increment, increment)
        raw = requests.get(url = url).json()
        output.extend(processingFunction(raw))
    return output

In [ ]:
get_history()

In [12]:
from collections import namedtuple

In [13]:
Listen = namedtuple('Listen', ['song_name', 'song_id', 'album_id', 'artist_id', "timestamp"], verbose=True)

from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Listen(tuple):
    'Listen(song_name, song_id, album_id, artist_id, timestamp)'

    __slots__ = ()

    _fields = ('song_name', 'song_id', 'album_id', 'artist_id', 'timestamp')

    def __new__(_cls, song_name, song_id, album_id, artist_id, timestamp):
        'Create new instance of Listen(song_name, song_id, album_id, artist_id, timestamp)'
        return _tuple.__new__(_cls, (song_name, song_id, album_id, artist_id, timestamp))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Listen object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 5:
            raise TypeError('Expected 5 arguments, got %d' % len(result))
        return result

    def _replace(_self, **kwds):
        'Return a new Listen object replacing specified fields with new values'


In [ ]:
example = Listen("Defector", "Drones", "Muse", 1)

In [ ]:
raw = get_history()

In [8]:
def process_history(data):
    return [Listen(item["title"], item["id"], item["album"]["id"], item["artist"]["id"], item["timestamp"]) for item in data["data"]]

In [14]:
longHistory = get_history(length = 1000)

In [ ]:
listenHistory = process_history(historyExample)

In [21]:
import numpy as np
%matplotlib inline
from IPython.core.pylabtools import figsize
from matplotlib import pyplot as plt
from collections import Counter

In [ ]:
timestamps = [x.timestamp for x in longHistory]
artists = [x.artist_name for x in longHistory]

In [ ]:
def display_pie_chart(listenHistory, number_items = 10):
    artists = [x.artist_name for x in listenHistory]
    counter = Counter(artists)
    
    artists, count = [x[0] for x in counter.most_common(number_items)], [x[1] for x in counter.most_common(number_items)]
    plt.pie(count, labels = artists)

In [ ]:
display_pie_chart(longHistory, number_items = 20 )

In [15]:
def most_listened_to_songs(listenHistory, number_items = 40):
    titles = [x.song_id for x in listenHistory]
    counter = Counter(titles)
    return [x[0] for x in counter.most_common(number_items)]

In [ ]:
most_listened_to_songs(longHistory)

In [16]:
class Playlist:
    
    def __init__(self, p_id = 1609467705,name = "JakesSuperAmazingFlow"):
        self.name = name
        self.url = r"http://api.deezer.com/playlist/{0}/tracks?access_token={1}".format(p_id,access_token)
        self.p_id = p_id

    def create(self):
        return False
        
    def add_tracks(self, ids):
        url = self.url + "&songs=" + self._parse_track_input(ids)
        return requests.post(url).json()
    
    def get_tracks(self):
        return requests.get(self.url).json()
    
    def remove_tracks(self, ids):
        url = self.url + "&songs=" + self._parse_track_input(ids)
        requests.delete(url)
        
    def clear_tracks(self):
        tracks = [x["id"] for x in playlist.get_tracks()["data"]]
        while tracks != []:
            self.remove_tracks(tracks)
            tracks = [x["id"] for x in playlist.get_tracks()["data"]]
    
    
    def _parse_track_input(self, ids):
        return "%2C".join([str(id) for id in ids])
    

In [22]:
import random

In [17]:
def select_from_generator(songList, amount):
    if len(songList) < amount: return songList
    random.shuffle(songList)
    return songList[:amount - 1]

def generate_flow(generators, history = longHistory, weights = None):
    
    flow_length = 50
    
    if weights == None: weights = [1] * len(generators)
        
    totalWeight = sum(weights)
        
    selectPerGenerator = [int(flow_length * (weight / totalWeight)) for weight in weights ] 
    
    print(selectPerGenerator)
    
    output = []
    
    for i in range(len(generators)):
        output.extend(select_from_generator(generators[i](history), selectPerGenerator[i]))
        
    
    return output

In [23]:
playlist = Playlist()
playlist.clear_tracks()
playlist.add_tracks(generate_flow([most_listened_to_songs, random_song_from_top_n_artists, deezer_flow]))

[16, 16, 16]


True

In [20]:
generate_flow([most_listened_to_songs, random_song_from_top_n_artists, deezer_flow])

[16, 16, 16]


NameError: name 'Counter' is not defined

In [18]:
def random_song_from_top_n_artists(history, number_of_artists = 5):
    artists = [x.artist_id for x in history]
    counter = Counter(artists)
    artist_ids = [x[0] for x in counter.most_common(5)]
    songs = []
    for artist_id in artist_ids:
        raw = requests.get(r"https://api.deezer.com/artist/{0}/top?limit=20".format(artist_id)).json()
        songs.extend([x["id"] for x in raw["data"]])
    return songs

In [19]:
def deezer_flow(history):
    url = r"http://api.deezer.com/user/me/flow?access_token={0}".format(access_token)
    return [x["id"] for x in requests.get(url).json()["data"]]

In [225]:
deezer_flow("Jake")

[15908435,
 2420458,
 95956384,
 13139232,
 910079,
 74683990,
 1178034,
 3129407,
 4622567,
 3157501,
 3590194,
 13547128,
 6685848,
 2924841,
 4624027,
 946317,
 3130433,
 2754843,
 102292706,
 44230981,
 1178161,
 67238732,
 39376531,
 2936707,
 7179758]

In [1]:
import theano